In [19]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
import scipy.sparse as sparse
import re
from sklearn.decomposition import TruncatedSVD
#from tensorflow.python.keras.preprocessing.text import Tokenizer

In [20]:
with open("data/cookingClean/Cooking_Train.pkl", 'rb') as f:
    df_train = pickle.load(f)
with open("data/cookingClean/Cooking_Test.pkl", 'rb') as f:
    df_test = pickle.load(f)
with open("data/cookingClean/Cooking_Valid.pkl", 'rb') as f:
    df_valid = pickle.load(f)




Making the Union of the labels because they are not the same in each sample

In [21]:
#labels = list(set(list(df_train)) | set(list(df_valid)) | set(list(df_test)))
labels = list(df_train)
labels.remove("text")
labels = ["text"] + labels

In [22]:
def fill_in_datasets(labs, df):    
    out = pd.DataFrame()
    for lab in labs :        
        if lab in list(df):
            out[lab] = df[lab]        
        else:
            out[lab] = (df.shape[0])*[0]
    return(out)

df_train = fill_in_datasets(labels, df_train)
df_test = fill_in_datasets(labels, df_test)
df_valid = fill_in_datasets(labels, df_valid)

labels = labels[1:]

## First model using TFIDF and logreg

In [23]:
import nltk.corpus 
nltk.download('stopwords')
stpW = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gonzales/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
transform_com = TfidfVectorizer(analyzer = 'char', ngram_range=(1,4), max_features=50000,stop_words=stpW ,min_df=2).fit(df_train.text)


%time comments_train = transform_com.transform(df_train.text)
%time comments_test = transform_com.transform(df_test.text)
%time comments_valid = transform_com.transform(df_valid.text)

CPU times: user 1.44 s, sys: 3.98 ms, total: 1.44 s
Wall time: 1.25 s
CPU times: user 160 ms, sys: 0 ns, total: 160 ms
Wall time: 160 ms
CPU times: user 150 ms, sys: 0 ns, total: 150 ms
Wall time: 150 ms


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

preds = np.zeros((df_test.shape[0], len(labels)))
model =  LogisticRegression(penalty = 'l2', C= 9.0)

for i, j in enumerate(labels[1:]):
    print('fit '+j)
    model.fit(comments_train, df_train[j])
    pred_train = model.predict_proba(comments_train)[:,1]
    print('Training loss: {}'.format(log_loss(df_train[j], pred_train)))
    preds[:,i] = model.predict_proba(comments_test)[:,1]
    gc.collect()

fit knife-safety
Training loss: 0.000760664502227679
fit mortar


/home/gonzales/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training loss: 0.0007934961639252384
fit grating


/home/gonzales/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training loss: 0.0008563642487704041
fit peel


...

Training loss: 0.0015269805679298317
fit tortilla-press


/home/gonzales/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training loss: 0.0004356061742523136
fit yeast


/home/gonzales/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training loss: 0.0076461094751632975


## Second modele using LSTM

In [ ]:
#PARAMETERS
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 500
embedding_vector_length = 100
batch_size = 128
nb_epochs = 10


comments = df_train.text.tolist() + df_test.text.tolist() + df_valid.text.tolist()
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(comments)

sequences_train = tokenizer.texts_to_sequences(df_train.comment_text)
sequences_test = tokenizer.texts_to_sequences(df_test.comment_text)
sequences_valid = tokenizer.texts_to_sequences(df_valid.comment_text)

x_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
x_valid = pad_sequences(sequences_valid, maxlen=MAX_SEQUENCE_LENGTH)


print('Shape of train tensor ', x_train.shape)
print('Shape of test tensor ', x_test.shape)
print('Shape of valid tensor ', x_valid.shape)

### A transformer en pytorch

In [ ]:
def create_model():
    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, embedding_vector_length, input_length=MAX_SEQUENCE_LENGTH))
    model.add(LSTM(100, return_sequences = True, dropout = 0.2))
    model.add(GlobalMaxPool1D())
    model.add(Dense(100,  activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(50,  activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model